<a href="https://colab.research.google.com/github/habiib1999/Speech-Emotion-Recognition/blob/main/emotionRecMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio
import os
import re

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Audio
# from entropy import spectral_entropy
from keras import layers
from keras import models
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import itertools

# CLASS : READER

In [8]:
#READER CLASS
class Reader:
  def __init__(self, crema_path):
    self.crema_path = crema_path


  def readCrema(self):
    emotion_df = []

    for wav in os.listdir(crema_path):
      info = wav.partition(".wav")[0].split("_")
      if info[2] == 'SAD':
        emotion_df.append(("sad", crema_path + "/" + wav))
      elif info[2] == 'ANG':
        emotion_df.append(("angry", crema_path + "/" + wav))
      elif info[2] == 'DIS':
        emotion_df.append(("disgust", crema_path + "/" + wav))
      elif info[2] == 'FEA':
        emotion_df.append(("fear", crema_path + "/" + wav))
      elif info[2] == 'HAP':
        emotion_df.append(("happy", crema_path + "/" + wav))
      elif info[2] == 'NEU':
        emotion_df.append(("neutral", crema_path + "/" + wav))
      else:
        emotion_df.append(("unknown", crema_path + "/" + wav))


    Crema_df = pd.DataFrame.from_dict(emotion_df)
    Crema_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)

    #print(Crema_df.head())
    return Crema_df

 

  def read(self):
    crema_dataset = self.readCrema()
    return crema_dataset
    

In [9]:
crema_path = "/content/drive/MyDrive/emotionDataset/Crema"
reader = Reader(crema_path)
crema_dataset = reader.read()
#from sklearn.utils import shuffle
#crema_dataset = shuffle(crema_dataset)
print(crema_dataset.head(10))

   Emotion                                               Path
0     fear  /content/drive/MyDrive/emotionDataset/Crema/10...
1     fear  /content/drive/MyDrive/emotionDataset/Crema/10...
2      sad  /content/drive/MyDrive/emotionDataset/Crema/10...
3  disgust  /content/drive/MyDrive/emotionDataset/Crema/10...
4  disgust  /content/drive/MyDrive/emotionDataset/Crema/10...
5    angry  /content/drive/MyDrive/emotionDataset/Crema/10...
6      sad  /content/drive/MyDrive/emotionDataset/Crema/10...
7  disgust  /content/drive/MyDrive/emotionDataset/Crema/10...
8     fear  /content/drive/MyDrive/emotionDataset/Crema/10...
9     fear  /content/drive/MyDrive/emotionDataset/Crema/10...


In [10]:
def energy(data, frame_length=2048, hop_length=512):
    en = np.array([np.sum(np.power(np.abs(data[hop:hop+frame_length]), 2)) for hop in range(0, data.shape[0], hop_length)])
    return en / frame_length
def mel_spc(data, sr, frame_length=2048, hop_length=512, flatten: bool = False):
    mel = librosa.feature.melspectrogram(y=data, sr=sr)
    return np.squeeze(mel.T) if not flatten else np.ravel(mel.T)

In [11]:
def extract_features(data, sr, frame_length=2048, hop_length=512):
    result = np.array([])

    return energy(data, frame_length, hop_length),mel_spc(data, sr, frame_length, hop_length)


def get_features(path, duration=2.5, offset=0.6):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=duration, offset=offset)

    energy_feature,mel_spec_feature = extract_features(data, sample_rate)

    return np.array(energy_feature) , np.array(mel_spec_feature)

In [12]:
path = np.array(crema_dataset["Path"])[657]
data, sampling_rate = librosa.load(path, duration=2.5, offset=0.6)
print(len(data))
print(data)
print(sampling_rate)
enrg = energy(data)
mel_spec = mel_spc(data, sampling_rate)
print("Energy shape: ", enrg.shape)
print("Energy: ", enrg)
print("MelSpectrogram shape: ", mel_spec.shape)
print("MelSpectrogram : ", mel_spec)

41215
[-0.00774202 -0.00921047 -0.00838774 ...  0.          0.
  0.        ]
22050
Energy shape:  (81,)
Energy:  [1.25416682e-05 9.00268205e-05 1.85334604e-04 3.20473308e-04
 4.43909288e-04 5.84825990e-04 6.93780254e-04 7.57424335e-04
 7.28843384e-04 5.22820163e-04 3.26876121e-04 2.08084093e-04
 3.37646983e-04 4.53232380e-04 6.37009856e-04 6.90810732e-04
 6.26968686e-04 6.22040359e-04 5.71376935e-04 5.53829479e-04
 5.40389272e-04 5.17104869e-04 4.33028355e-04 3.69260844e-04
 3.49385751e-04 6.80433295e-04 9.92599176e-04 1.06770382e-03
 1.07393204e-03 7.50555773e-04 3.96377407e-04 3.65105516e-04
 3.75498668e-04 3.81877937e-04 4.22568090e-04 4.03064187e-04
 3.02504777e-04 2.81216606e-04 3.60311329e-04 4.31530643e-04
 5.63004171e-04 7.37200084e-04 9.54805641e-04 1.11985242e-03
 1.10804976e-03 1.04999903e-03 7.55362853e-04 4.82189003e-04
 3.56042641e-04 1.70072773e-04 1.18846765e-04 8.40425055e-05
 5.14617059e-05 3.80887905e-05 3.09384013e-05 2.71017125e-05
 2.12698069e-05 1.17438049e-05 1.

In [13]:
extracted_data_energy, extracted_data_mel_spec = [], []
extracted_data_energy_labels, extracted_data_mel_spec_labels = [], []
print("Feature processing...")
for path, emotion, ind in zip(crema_dataset.Path, crema_dataset.Emotion, range(crema_dataset.Path.shape[0])):
    energy_feature,mel_spec_feature = get_features(path)
    if ind % 100 == 0:
        print(f"{ind} samples has been processed...")
    extracted_data_energy.append(energy_feature)      
    extracted_data_energy_labels.append(emotion)
    extracted_data_mel_spec.append(mel_spec_feature)      
    extracted_data_mel_spec_labels.append(emotion)

print("Done.")

Feature processing...
0 samples has been processed...
100 samples has been processed...
200 samples has been processed...
300 samples has been processed...
400 samples has been processed...
500 samples has been processed...
600 samples has been processed...
700 samples has been processed...
800 samples has been processed...
900 samples has been processed...
1000 samples has been processed...
1100 samples has been processed...
1200 samples has been processed...
1300 samples has been processed...
1400 samples has been processed...
1500 samples has been processed...
1600 samples has been processed...
1700 samples has been processed...
1800 samples has been processed...
1900 samples has been processed...
2000 samples has been processed...
2100 samples has been processed...
2200 samples has been processed...
2300 samples has been processed...
2400 samples has been processed...
2500 samples has been processed...
2600 samples has been processed...
2700 samples has been processed...
2800 sampl

In [14]:
features_path = "/content/drive/MyDrive/emotionDataset/crema_energy_features.csv"
extracted_df = pd.DataFrame(extracted_data_energy)
extracted_df["labels"] = extracted_data_energy_labels
extracted_df.to_csv(features_path, index=False)
extracted_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,labels
0,0.000157,0.000105,0.000075,0.000068,0.000072,0.000094,0.000273,0.000587,0.000858,0.000934,0.000844,0.000561,0.000283,0.000161,0.000065,0.000031,0.000030,0.000030,0.000031,0.000027,0.000020,0.000018,0.000019,0.000023,0.000027,0.000028,0.000025,0.000022,0.000022,0.000020,0.000018,0.000019,0.000021,0.000019,0.000021,0.000019,0.000017,0.000025,0.000025,0.000028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fear
1,0.012323,0.066642,0.094322,0.124989,0.130533,0.095000,0.077821,0.052947,0.041790,0.023789,0.012663,0.017132,0.018025,0.034618,0.043925,0.035394,0.032040,0.013879,0.005472,0.004253,0.003301,0.003832,0.004409,0.004036,0.004221,0.003986,0.002571,0.001963,0.001194,0.001412,0.003720,0.015239,0.028289,0.035175,0.046813,0.050180,0.051656,0.053262,0.042210,0.044249,...,0.000027,0.000032,0.000028,0.000034,0.000037,0.000029,0.000023,0.000013,0.000004,6.883546e-07,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fear
2,0.000680,0.000572,0.000464,0.000555,0.000648,0.000767,0.000781,0.000633,0.000516,0.000511,0.000635,0.000630,0.000627,0.000474,0.000209,0.000141,0.000296,0.000562,0.000884,0.001068,0.001003,0.000909,0.000677,0.000495,0.000351,0.000158,0.000070,0.000129,0.000166,0.000279,0.000328,0.000316,0.000305,0.000224,0.000193,0.000149,0.000113,0.000083,0.000053,0.000036,...,0.000028,0.000032,0.000030,0.000028,0.000025,0.000025,0.000026,0.000026,0.000027,2.649021e-05,0.000023,0.000024,0.000025,0.000020,0.000024,0.000024,0.000019,0.000014,0.000007,0.000002,3.974998e-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sad
3,0.000021,0.000026,0.000032,0.000034,0.000030,0.000026,0.000024,0.000019,0.000020,0.000018,0.000018,0.000026,0.000033,0.000043,0.000061,0.000082,0.000127,0.000162,0.000182,0.000190,0.000172,0.000162,0.000156,0.000142,0.000130,0.000122,0.000149,0.000217,0.000315,0.000500,0.000521,0.000484,0.000435,0.000281,0.000402,0.000575,0.000892,0.001104,0.001055,0.001084,...,0.000079,0.000063,0.000058,0.000088,0.000107,0.000107,0.000102,0.000064,0.000042,4.238462e-05,0.000057,0.000094,0.000138,0.000180,0.000204,0.000200,0.000183,0.000147,0.000132,0.000122,1.127197e-04,0.000127,0.000106,0.000088,0.000069,0.000036,0.000030,0.000023,0.000025,0.000028,0.000024,0.000031,0.000035,0.000034,0.000036,0.000027,0.000015,0.000008,0.000002,disgust
4,0.000025,0.000024,0.000033,0.000039,0.000059,0.000077,0.000087,0.000095,0.000098,0.000093,0.000101,0.000127,0.000126,0.000117,0.000093,0.000058,0.000063,0.000084,0.000085,0.000086,0.000075,0.000068,0.000108,0.000140,0.000154,0.000178,0.000184,0.000444,0.000713,0.000993,0.001350,0.001468,0.001649,0.001925,0.002276,0.002866,0.003592,0.003995,0.003897,0.003309,...,0.000267,0.000562,0.000828,0.000809,0.000691,0.000582,0.000680,0.000934,0.002013,2.552214e-03,0.003398,0.003889,0.003068,0.002526,0.001461,0.000749,0.000508,0.000467,0.000421,0.000401,3.569985e-04,0.000248,0.000166,0.000116,0.000100,0.000071,0.000059,0.000047,0.000041,0.000043,0.000045,0.000044,0.000041,0.000032,0.000025,0.000023,0.000011,0.000007,0.000004,disgust


In [15]:
features_path = "/content/drive/MyDrive/emotionDataset/crema_mel_spec_features.csv"
extracted_df = pd.DataFrame(extracted_data_mel_spec)
extracted_df["labels"] = extracted_data_mel_spec_labels
extracted_df.to_csv(features_path, index=False)
extracted_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


,0,labels
0,"[[0.3735934, 0.6247544, 1.9075246, 2.1719446, ...",fear
1,"[[0.110143304, 0.01234068, 0.010340791, 0.1479...",fear
2,"[[0.09649463, 0.17921183, 0.085100114, 0.18549...",sad
3,"[[0.17824107, 0.034192763, 0.040228058, 0.0265...",disgust
4,"[[0.04045482, 0.003997328, 0.030719493, 0.0737...",disgust


In [16]:
features_path = "/content/drive/MyDrive/emotionDataset/crema_energy_features.csv"
energy_data = pd.read_csv(features_path)
energy_data.head()
print(energy_data)
print(energy_data.shape)

             0         1         2  ...       106       107   labels
0     0.000157  0.000105  0.000075  ...       NaN       NaN     fear
1     0.012323  0.066642  0.094322  ...       NaN       NaN     fear
2     0.000680  0.000572  0.000464  ...       NaN       NaN      sad
3     0.000021  0.000026  0.000032  ...  0.000008  0.000002  disgust
4     0.000025  0.000024  0.000033  ...  0.000007  0.000004  disgust
...        ...       ...       ...  ...       ...       ...      ...
7437  0.000016  0.000021  0.000033  ...  0.000009  0.000004  disgust
7438  0.000018  0.000019  0.000022  ...  0.000020  0.000006  neutral
7439  0.000043  0.000055  0.000069  ...       NaN       NaN  disgust
7440  0.023445  0.024009  0.013516  ...  0.000020  0.000007    angry
7441  0.005099  0.014057  0.016144  ...  0.000022  0.000003    angry

[7442 rows x 109 columns]
(7442, 109)


In [18]:
features_path = "/content/drive/MyDrive/emotionDataset/crema_mel_spec_features.csv"
mel_spec_data = pd.read_csv(features_path)
mel_spec_data.head()
print(mel_spec_data)
print(len(mel_spec_data.loc[0][0]))

                                                      0   labels
0     [[3.73593390e-01 6.24754429e-01 1.90752459e+00...     fear
1     [[1.10143304e-01 1.23406798e-02 1.03407912e-02...     fear
2     [[9.6494630e-02 1.7921183e-01 8.5100114e-02 .....      sad
3     [[1.7824107e-01 3.4192763e-02 4.0228058e-02 .....  disgust
4     [[4.04548198e-02 3.99732823e-03 3.07194926e-02...  disgust
...                                                 ...      ...
7437  [[8.1536090e-03 4.9170428e-03 2.2856034e-03 .....  disgust
7438  [[8.93165916e-03 2.35754643e-02 2.67566908e-02...  neutral
7439  [[2.57985473e-01 1.21228591e-01 7.34253973e-03...  disgust
7440  [[6.95406944e-02 2.77917944e-02 7.31072295e-03...    angry
7441  [[6.52315887e-03 1.59181282e-02 1.01051880e-02...    angry

[7442 rows x 2 columns]
599
